In [31]:

import collections, functools, operator
import numpy as np
import pandas as pd
import operator
import re
import os 

import praw
import emoji
from concurrent.futures import ThreadPoolExecutor, as_completed

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
COMMON_WORDS = set(stopwords.words("english"))

In [2]:
class Get_subreddit():
    def __init__(self, subreddit):
        reddit = praw.Reddit(
            client_id= os.environ['CLIENT_ID_REDDIT'], 
            client_secret= os.environ['CLIENT_SECRET_REDDIT'], 
            user_agent="hsabbar",
        )
        self.hot = reddit.subreddit(subreddit).hot(limit=500)
        self.max_comments = 100
        self.data = []
        self.columns = ['title','id','score', 'date', 'author', 'content', 'num_comments', 'comments'] 
        
    def get_data(self, submission):
        data_tmp = []
        comment_data = []
        data_tmp = [submission.title, submission.id, submission.score,submission.created_utc, submission.author, 
                     submission.selftext, len(submission.comments)]

        submission.comments.replace_more(limit=self.max_comments)
        comment_data = [top_level_comment.body for top_level_comment in submission.comments[:self.max_comments]]
        

        """
        for top_level_comment in submission.comments[:self.max_comments]: 
            comment_data.append(top_level_comment.body)
        if len(comment_data) < self.max_comments:
            comment_data = comment_data + [None] * (self.max_comments - len(comment_data))
        """
        return data_tmp + [' '.join(comment_data)]
    
    def get_data_multi_thread(self):
        processes = []
        ccccccc

        self.data = [task.result() for task in as_completed(processes)]
        
        #print(self.data)
        return pd.DataFrame(self.data, columns = self.columns)

In [80]:
%%time
reddit_data = Get_subreddit("Bitcoin")
df = reddit_data.get_data_multi_thread()
df.to_csv("subReddit-data-Bitcion.csv", sep=';', encoding='utf-8')
#df.head()

Wall time: 3.01 s


In [3]:
df2 = pd.read_csv("subReddit-data-Bitcion.csv", sep=';')
df2.head()

,Unnamed: 0,title,id,score,date,author,content,num_comments,comments
0,0,Best hardware wallet?,nla5z5,16,1.622011e+09,Negative_Comedian870,Hey guys. \n\nSeeing lots of people on the Led...,16,"I personaly use trezor, but i think the only p..."
1,1,"IT Giant Globant Joins Buys $500,000 Worth of ...",nkmhp2,36,1.621939e+09,VAMPXIII,NaN,10,People here acting like 10 BTC is a small quan...
2,2,Cryptocurrency Had a Dire Weekend. Why?,nl3nd2,1,1.621988e+09,SnooSketches3367,NaN,1,Novagrass ? talked about 5 factors on utube's ...
3,3,Wash-sale regulation doesn't apply to Bitcoin ...,nlra50,1,1.622063e+09,Wishy_washy_Though,"""Bitcoin crash opens door to a tax loophole fo...",1,"Bob buys 1 BTC at $60K. Later, Bob sells 1 BTC..."
4,4,Bitcoin Surges Over $40k After Bitcoin Mining ...,nlkf4q,0,1.622045e+09,crypto_pub,NaN,5,Yea no Bitcoin would test 40k no matter the la...


In [4]:
class Data_processing():
    def __init__(self, df):
        self.df = df
        self.columns_text = ['title', 'content', 'comments'] #+ [f'comment{i}' for i in range(1, 101)]
        self.del_url = re.compile(r'https(.*?)(\s|$)')
    
    
    def my_stopwords(self, text):
        text_tokens = word_tokenize(text)
        data = [word for word in text_tokens if word not in COMMON_WORDS and len(word) > 1]
        return " ".join(data)

    
        
    def nlp_pipeline(self, text):
        if text is not None :
            
            text = str(text).lower()
            #text = text.replace('\n', ' ').replace('\r', '')
            #text = ' '.join(text.split())
            #text = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", ext)
            #text = re.sub(r"(\s\-\s|-$)", "", text)
            #text = re.sub(r"\&\S*\s", " ", text)
            #text = re.sub(r"['&@¨=“”’‘+—^.;:-_~!-/\{\}*=\%?,#<>\"()\[\]]", "", text)
            
            #for url in self.del_url.findall(text):
             #   text = re.sub(f'https{url[0]}', "", text)
            text = emoji.get_emoji_regexp().sub("", text) 
            
            #text = self.my_stopwords(text)
            return text
        else :
            pass
    
    def clean_data(self):
        for col_txt in self.columns_text :
            #print(col_txt)
            self.df[col_txt] = self.df[col_txt].apply(lambda x:  self.nlp_pipeline(x))
        
        return self.df

In [5]:
%%time
data_proc = Data_processing(df2)
df2 = data_proc.clean_data()
df2.head()

Wall time: 7.68 s


,Unnamed: 0,title,id,score,date,author,content,num_comments,comments
0,0,best hardware wallet?,nla5z5,16,1.622011e+09,Negative_Comedian870,hey guys. \n\nseeing lots of people on the led...,16,"i personaly use trezor, but i think the only p..."
1,1,"it giant globant joins buys $500,000 worth of ...",nkmhp2,36,1.621939e+09,VAMPXIII,nan,10,people here acting like 10 btc is a small quan...
2,2,cryptocurrency had a dire weekend. why?,nl3nd2,1,1.621988e+09,SnooSketches3367,nan,1,novagrass ? talked about 5 factors on utube's ...
3,3,wash-sale regulation doesn't apply to bitcoin ...,nlra50,1,1.622063e+09,Wishy_washy_Though,"""bitcoin crash opens door to a tax loophole fo...",1,"bob buys 1 btc at $60k. later, bob sells 1 btc..."
4,4,bitcoin surges over $40k after bitcoin mining ...,nlkf4q,0,1.622045e+09,crypto_pub,nan,5,yea no bitcoin would test 40k no matter the la...


In [6]:
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment.util import *

from nltk import tokenize

In [7]:
class Label_nltk():
    def __init__(self, df):
        self.df = df
        self.colmuns_text = ["title", "content", "comments"]
        
    def label(self, colmun):
        labels = []
        for col in self.df.colmun:
            suntances = tokenize.sent_tokenize(col)
            #label_last = None
            for suntance in suntances:
                scor = sia.polarity_scores(suntance)
                label = max(scor.items(), key=operator.itemgetter(1))

                #if label == label_last :
                #    print(label, suntance)
                #label_last = label
            labels.append(label)
        self.df[f'label_{colmun}'] = labels
        
    def make_lable(self):
        with ThreadPoolExecutor(max_workers=62) as executor:
            for colmun in self.colmuns_text:
                executor.submit(self.label, colmun)

In [8]:
%%time
data_labels = Label_nltk(df2)
data_labels.make_lable()
df2.head()

Wall time: 8.14 ms


,Unnamed: 0,title,id,score,date,author,content,num_comments,comments
0,0,best hardware wallet?,nla5z5,16,1.622011e+09,Negative_Comedian870,hey guys. \n\nseeing lots of people on the led...,16,"i personaly use trezor, but i think the only p..."
1,1,"it giant globant joins buys $500,000 worth of ...",nkmhp2,36,1.621939e+09,VAMPXIII,nan,10,people here acting like 10 btc is a small quan...
2,2,cryptocurrency had a dire weekend. why?,nl3nd2,1,1.621988e+09,SnooSketches3367,nan,1,novagrass ? talked about 5 factors on utube's ...
3,3,wash-sale regulation doesn't apply to bitcoin ...,nlra50,1,1.622063e+09,Wishy_washy_Though,"""bitcoin crash opens door to a tax loophole fo...",1,"bob buys 1 btc at $60k. later, bob sells 1 btc..."
4,4,bitcoin surges over $40k after bitcoin mining ...,nlkf4q,0,1.622045e+09,crypto_pub,nan,5,yea no bitcoin would test 40k no matter the la...


In [33]:
%%time
def label(df):
    labels = []
    sia = SentimentIntensityAnalyzer()
    for col in df.comments: 
        paragraph = tokenize.sent_tokenize(str(col))
        print('-------------')
        for suntance in paragraph:
            scor = sia.polarity_scores(suntance)
            #label = max(scor.items(), key=operator.itemgetter(1))
            print(scor)
            label = dict(functools.reduce(operator.add, map(collections.Counter, scor)))
            print(label)
        labels.append(label)
        
    df[f'label_comments'] = labels
    
    
label(df2)
df2.head()

-------------
{'neg': 0.182, 'neu': 0.818, 'pos': 0.0, 'compound': -0.5499}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.261, 'neu': 0.739, 'pos': 0.0, 'compound': -0.5574}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.61, 'neu': 0.39, 'pos': 0.0, 'compound': -0.5719}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.123, 'neu': 0.877, 'pos': 0.0, 'compound': -0.1027}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.432, 'pos': 0.568, 'compound': 0.8591}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.318, 'neu': 0.682, 'pos': 0.0, 'compound': -0.681}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 

{'neg': 0.0, 'neu': 0.421, 'pos': 0.579, 'compound': 0.6705}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.65, 'pos': 0.35, 'compound': 0.8979}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.256, 'pos': 0.744, 'compound': 0.4404}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.073, 'neu': 0.927, 'pos': 0.0, 'compound': -0.0857}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.094, 'neu': 0.695, 'pos': 0.211, 'compound': 0.5719}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.156, 'neu

{'neg': 0.063, 'neu': 0.937, 'pos': 0.0, 'compound': -0.3559}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.625, 'pos': 0.375, 'compound': 0.4588}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.037, 'neu': 0.759, 'pos': 0.204, 'compound': 0.8355}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.055, 'neu': 0.473, 'pos': 0.473, 'compound': 0.9501}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.891, 'neu': 0.109, 'pos': 0.0, 'compound': -0.8481}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.131, 'neu': 0.765, 'pos': 0.104, 'compound': -0.1295}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'ne

{'neg': 0.0, 'neu': 0.872, 'pos': 0.128, 'compound': 0.3612}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.755, 'pos': 0.245, 'compound': 0.6369}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound': 0.3612}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.161, 'neu': 0.839, 'pos': 0.0, 'compound': -0.5569}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.861, 'po

{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.726, 'pos': 0.274, 'compound': 0.7506}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.776, 'pos': 0.224, 'compound': 0.7469}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.109, 'neu': 0.806, 'pos': 0.085, 'compound': -0.1531}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.847, 'pos': 0.153, 'compound': 0.4404}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.276, 'neu': 0.724, 'pos': 0.0, 'compound': -0.7906}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.736, 'pos': 0.264, 'compound': 0.7845}
{'n': 3, 

...

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.853, 'pos': 0.147, 'compound': 0.2263}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.874, 'pos': 0.126, 'compound': 0.3182}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.251, 'neu': 0.627, 'pos': 0.123, 'compound': -0.6908}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.068, 'neu': 0.583, 'pos': 0.35, 'compound': 0.8271}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0

{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.843, 'pos': 0.157, 'compound': 0.3818}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.224, 'neu': 0.641, 'pos': 0.135, 'compound': -0.34}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.251, 'neu': 0.749, 'pos': 0.0, 'compound': -0.5719}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.091, 'neu': 0.783, 'pos': 0.125, 'compound': 0.0772}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.388, 'neu': 0.402, 'pos': 0.209, 'compound': -0.3022}
{'n': 3, 'e':

{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.928, 'pos': 0.072, 'compound': 0.3612}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.636, 'neu': 0.364, 'pos': 0.0, 'compound': -0.5423}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.815, 'pos': 0.185, 'compound': 0.3612}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound': 0.4588}
{'n': 3, 'e': 2, 'g': 1, 'u': 

{'neg': 0.076, 'neu': 0.703, 'pos': 0.222, 'compound': 0.4019}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.611, 'pos': 0.389, 'compound': 0.6114}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound': 0.4585}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'compound': 0.7227}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4215}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.0, 'neu': 0.851, 'pos': 0.149, 'compound': 0.2732}
{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': 3, 's': 1, 'c': 1, 'm': 1, 'd': 1}
{'neg': 0.181, 'neu

,Unnamed: 0,title,id,score,date,author,content,num_comments,comments,label_comments
0,0,best hardware wallet?,nla5z5,16,1.622011e+09,Negative_Comedian870,hey guys. \n\nseeing lots of people on the led...,16,"i personaly use trezor, but i think the only p...","{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': ..."
1,1,"it giant globant joins buys $500,000 worth of ...",nkmhp2,36,1.621939e+09,VAMPXIII,nan,10,people here acting like 10 btc is a small quan...,"{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': ..."
2,2,cryptocurrency had a dire weekend. why?,nl3nd2,1,1.621988e+09,SnooSketches3367,nan,1,novagrass ? talked about 5 factors on utube's ...,"{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': ..."
3,3,wash-sale regulation doesn't apply to bitcoin ...,nlra50,1,1.622063e+09,Wishy_washy_Though,"""bitcoin crash opens door to a tax loophole fo...",1,"bob buys 1 btc at $60k. later, bob sells 1 btc...","{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': ..."
4,4,bitcoin surges over $40k after bitcoin mining ...,nlkf4q,0,1.622045e+09,crypto_pub,nan,5,yea no bitcoin would test 40k no matter the la...,"{'n': 3, 'e': 2, 'g': 1, 'u': 2, 'p': 2, 'o': ..."
